In [1]:
from pathlib import Path

In [2]:
VIDPATH = Path("/sbtal/riksdag-video/")

In [3]:
_SWE_MODEL = "jimregan/wav2vec2-swedish-phonetic-waxholm"

In [4]:
from transformers import pipeline
pipe = pipeline(model=_SWE_MODEL, device=1)

/Users/joregan/opt/anaconda3/envs/hf_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/joregan/opt/anaconda3/envs/hf_new/lib/python3.9/site-packages/huggingface_hub-0.16.4-py3.8.egg/huggingface_hub/utils/_hf_folder.py:95: UserWarning: A token has been found in `/Users/joregan/.huggingface/token`. This is the old path where tokens were stored. The new location is `/Users/joregan/.cache/huggingface/token` which is configurable using `HF_HOME` environment variable. Your token has been copied to this new location. You can now safely delete the old token file manually or use `huggingface-cli logout`.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [9]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(_SWE_MODEL)
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}


In [12]:
from pydub import AudioSegment

In [13]:
def convert_audio(audio_id):
    parameters=["-ac", "1", "-acodec", "pcm_s16le", "-ar", "16000"]
    video_path = VIDPATH / f"{audio_id}_480p.mp4"
    video = AudioSegment.from_file(str(video_path), "mp4")
    outname = f"/tmp/{audio_id}.wav"
    video.export(outname, format="wav", parameters=parameters)
    return outname

In [23]:
import json

In [24]:
JSON_PATH = Path("/home/joregan/rd_phonetic")

In [25]:
with open(str(JSON_PATH / f"{audid}.json"), "w") as jsonf:
    json.dump(output, jsonf)

In [30]:
for mapping in FILE_MAPPING.items():
    lmid, audio = mapping
    OUTFILE = JSON_PATH / f"{audio}.json"
    if not OUTFILE.exists():
        print(mapping)
        wavfile = convert_audio(audio)
        decoder = build_decoder(lmid)
        output = pipe(wavfile, chunk_length_s=10, return_timestamps="word", decoder=decoder)
        with open(str(OUTFILE), "w") as jsonf:
            json.dump(output, jsonf)

In [37]:
outputs = set([y for x, y in FILE_MAPPING.items()])
len(outputs)

396

In [38]:
reverse = {}
with open("mappings.tsv", "w") as mappings:
    for api, audio in FILE_MAPPING.items():
        if audio in reverse:
            continue
        reverse[audio] = api
        mappings.write(f"{api}\t{audio}\n")
